## Big Data e Processamento Otimizado com Python

### Lazy Evaluation vs. Eager Execution

Você já notou que o Pandas, às vezes, demora muito apenas para carregar um arquivo comparado ao Polars, correto? Isso acontece porque ele usa Eager Execution ("Execução Ansiosa"). O Pandas tenta carregar tudo para a memória RAM de uma vez e processar linha por linha imediatamente.

O Lazy Evaluation (usado pelo Polars e Spark) é diferente. Quando você dá um comando, a ferramenta não faz nada com os dados ainda. Ela apenas "anota" o pedido em um Plano de Execução. O processamento real só acontece quando você exige a impressão do resultado final (usando .collect()).

A tríade para este processamento do Polars é:

- **scan_parquet() ou .lazy()**: Ativa o "modo preguiçoso".

- **explain()**: Mostra o plano de execução (como o Polars pretende resolver sua consulta).

- **collect()**: O gatilho. É aqui que o processamento realmente ocorre e a memória é utilizada.

In [50]:
import polars as pl
import time

# 1. Modo Lazy: Estamos apenas escaneando o arquivo, sem carregar para a RAM
# scan_parquet já inicia em modo lazy por padrão.
q = (
    pl.scan_parquet("../Aula12/202501_NovoBolsaFamilia_POLARS.parquet")
    .rename({
        'M�S COMPET�NCIA': 'MES_COMPETENCIA',
        'M�S REFER�NCIA': 'MES_REFERENCIA',
        'C�DIGO MUNIC�PIO SIAFI': 'CODIGO_MUNICIPIO_SIAFI',
        'NOME MUNIC�PIO': 'NOME_MUNICIPIO',
    })
    .filter(pl.col("UF") == "SP")
)

print(type(q)) # Note que é um LazyFrame, não um DataFrame
print("\n")

# 2. O Plano de Execução (Explain)
print("--- Plano de Execução (Explain) ---")
print("O Polars analisou sua query e otimizou o caminho antes de tocar nos dados:")
print(q.explain())
print("\n")

# 3. Execução Real (Collect)
print("--- Executando (Collect) ---")
start_time = time.time()
df_resultado = q.collect()
end_time = time.time()

display(df_resultado)

<class 'polars.lazyframe.frame.LazyFrame'>


--- Plano de Execução (Explain) ---
O Polars analisou sua query e otimizou o caminho antes de tocar nos dados:
RENAME
  Parquet SCAN [../Aula12/202501_NovoBolsaFamilia_POLARS.parquet] [id: 5157118671952]
  PROJECT */9 COLUMNS
  SELECTION: [(col("UF")) == ("SP")]


--- Executando (Collect) ---


MES_COMPETENCIA,MES_REFERENCIA,UF,CODIGO_MUNICIPIO_SIAFI,NOME_MUNICIPIO,CPF FAVORECIDO,NIS FAVORECIDO,NOME FAVORECIDO,VALOR PARCELA
i64,i64,str,i64,str,str,i64,str,str
202501,202308,"""SP""",7071,"""SANTOS""","""***.085.106-**""",20643890445,"""FERNANDA RAMOS TEIXEIRA""","""650,00"""
202501,202309,"""SP""",7071,"""SANTOS""","""***.085.106-**""",20643890445,"""FERNANDA RAMOS TEIXEIRA""","""650,00"""
202501,202310,"""SP""",7071,"""SANTOS""","""***.085.106-**""",20643890445,"""FERNANDA RAMOS TEIXEIRA""","""650,00"""
202501,202311,"""SP""",7071,"""SANTOS""","""***.085.106-**""",20643890445,"""FERNANDA RAMOS TEIXEIRA""","""650,00"""
202501,202312,"""SP""",7071,"""SANTOS""","""***.085.106-**""",20643890445,"""FERNANDA RAMOS TEIXEIRA""","""650,00"""
…,…,…,…,…,…,…,…,…
202501,202501,"""SP""",2973,"""ZACARIAS""","""""",20491700983,"""VANDERSON DIOGO DA SILVA""","""600,00"""
202501,202501,"""SP""",2973,"""ZACARIAS""","""***.696.978-**""",16528891096,"""VANILDE ARMINDO DOS SANTOS""","""600,00"""
202501,202501,"""SP""",2973,"""ZACARIAS""","""***.960.048-**""",16583160529,"""WELINGTON LUCAS BATISTA COELHO""","""450,00"""


#### O que observar no explain()?
Procure por termos como SELECTION ou FILTER. Se o Polars perceber que você só quer dados de "SP", ele tentará ler do disco apenas as linhas de SP, ignorando o resto. Isso é chamado de **Predicate Pushdown**. No modelo **Eager** ("ansioso"), ele leria tudo para a memória e só depois filtraria.

### Análise Estatística e Desafios de Escala

Ao trabalhar com Big Data, um simples cálculo de média pode estourar a memória do seu computador se não for feito corretamente.

Vamos realizar uma análise estatística descritiva (média, desvio padrão, quartis) usando Lazy:

In [51]:
# Definindo a consulta estatística
stats_query = (
    pl.scan_parquet("../Aula12/202501_NovoBolsaFamilia_POLARS.parquet")
    .with_columns(
        pl.col("VALOR PARCELA")
        .str.replace(",", ".")     # "650,00" vira "650.00"
        .cast(pl.Float64)          # vira o número 650.0
    )
    .select([
        pl.col("VALOR PARCELA").min().alias("minimo"),
        pl.col("VALOR PARCELA").mean().alias("media"),
        pl.col("VALOR PARCELA").std().alias("desvio_padrao"),
        pl.col("VALOR PARCELA").max().alias("maximo"),
        pl.col("VALOR PARCELA").quantile(0.5).alias("mediana")
    ])
)

# Executando
print("--- Estatísticas Descritivas ---")
stats_df = stats_query.collect()
display(stats_df)

--- Estatísticas Descritivas ---


minimo,media,desvio_padrao,maximo,mediana
f64,f64,f64,f64,f64
25.0,670.023784,190.890276,3938.0,650.0


Quando usamos collect(), o Polars tenta paralelizar o trabalho em todos os núcleos da sua CPU. Se o dataset fosse maior que sua memória RAM, você poderia usar o *collect(streaming=True)*, que processa os dados em pedaços (chunks), permitindo analisar gigabytes de dados em um computador comum.

### Atividade Guiada de Negócios

Imagine que somos analistas de dados do governo. Recebemos dados de pagamentos e precisamos responder:

"Qual é a distribuição dos valores pagos por estado?"

"Existem estados com média de pagamento muito superior aos outros?"

Tente escrever uma consulta usando lazy() que agrupe por UF, calcule a média e a contagem de beneficiários, e ordene pelo maior valor médio.

(Tente fazer antes de olhar a resposta abaixo!)

In [52]:
analise_uf = (
    pl.scan_parquet("../Aula12/202501_NovoBolsaFamilia_POLARS.parquet")
    .with_columns(
        pl.col("VALOR PARCELA")
        .str.replace(",", ".")   
        .cast(pl.Float64)          
    )
    .group_by("UF")
    .agg([
        pl.col("VALOR PARCELA").mean().alias("valor_medio"),
        pl.col("VALOR PARCELA").count().alias("total_beneficiarios")
    ])
    .sort("valor_medio", descending=True)
)

display(analise_uf.collect())

UF,valor_medio,total_beneficiarios
str,f64,u32
"""RR""",736.644405,80181
"""AM""",726.921172,645901
"""AC""",718.560571,131969
"""AP""",717.468446,122379
"""PA""",697.382422,1347871
…,…,…
"""RJ""",659.954202,1577414
"""PR""",659.654456,597961
"""BA""",659.531868,2466527


### Atividade Prática

- Utilize sua base de dados do projeto (etapa 2) e crie 3 perguntas estratégicas no mesmo molde da atividade guiada acima;
- Responda essas perguntas por filtros e consultas dentro do Lazyframe.